In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np

# Релиз 2.0

In [2]:
path = 'https://raw.githubusercontent.com/AlexeyKamaev/test/main/_data.csv'
df = pd.read_csv(path)
pd.set_option('display.max_columns', None)

In [3]:
df = df.rename(columns={'Площадь, м2':'Площадь'}).drop('Ссылка на объявление',axis = 1).drop('Unnamed: 0',axis = 1).drop('Телефоны',axis = 1)
df = df[df['Адрес'].str.contains('Москва')]
df = df.dropna(subset=['Метро'])

In [4]:
def get_price(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  return int(r.split()[0])

def get_deposit(y):
  r = ''
  for i in y:
    if i.isdigit():
      r += i
    else:
      r += ' '
  try:
      if int(r.split()[2]) < 1000 or int(r.split()[2]) > int(r.split()[0]) * 6:
        return int(r.split()[0])
      else:
        return int(r.split()[2])
  except:
      return int(r.split()[0])
    
x1 = df['Цена'].map(get_price) 
x2 = df['Цена'].map(get_deposit) 

In [5]:
df.insert(9, "Цена за месяц",x1)
df.insert(10,'Залог',x2)

In [6]:
def first_element(x):
    y = re.findall(r'\d+', x)
    return y[0]

df['Этаж'] = df['Дом'].map(first_element).astype('int')
def second_element(x):
    y = re.findall(r'\d+', x)
    return y[1]

df['Этажность дома'] = df['Дом'].apply(second_element).astype('int')
from operator import methodcaller
def func(x):
  return float(x[0])
point_place = df['Площадь'].map(methodcaller("split", "/")).map(func).copy()
df.insert(5, "Общая площадь",point_place)

In [7]:
def func (y):
    y[:y.find('(')]
    return y[:y.find('(')].strip()

def func2 (y):
    y[:y.find('(')]
    return y[y.find('м. ')+3:].strip()

x = df['Метро'].map(func).map(func2).copy()
df.insert(3, "Станция", x)

In [8]:
#чистка
df.drop(['Метро', 'Цена','Описание','Дополнительно','Название ЖК'], axis= 1, inplace=True)
df = df[['ID  объявления','Адрес','Станция','Серия дома', 'Общая площадь',	'Площадь', 'Этаж',	'Этажность дома', 'Высота потолков, м', 'Площадь комнат, м2', 'Количество комнат','Ремонт','Балкон', 'Окна','Санузел','Лифт','Мусоропровод', 'Тип','Дом',	'Парковка',	'Цена за месяц','Залог','Можно с детьми/животными']].copy()
df = df.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True)

In [9]:
df.drop(['Лифт', 'Санузел', 'Высота потолков, м', 'Балкон', 'Парковка', 'Мусоропровод', 'Окна'], axis= 1, inplace=True)

In [10]:
df.head()

,ID,Адрес,Станция,Серия дома,Общая площадь,Площадь,Этаж,Этажность дома,"Площадь комнат, м2",Количество комнат,Ремонт,Тип,Дом,Цена за месяц,Залог,Можно с детьми/животными
0,271271157,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/20.0,5,16,NaN,4,Дизайнерский,Квартира,"5/16, Монолитный",500000,500000,"Можно с детьми, Можно с животными"
1,271634126,"Москва, улица Новый Арбат, 27",Смоленская,NaN,198.0,198.0/95.0/18.0,5,16,25 25 20 25,4,Дизайнерский,Квартира,"5/16, Монолитно-кирпичный",500000,500000,Можно с детьми
2,271173086,"Москва, улица Новый Арбат, 27",Смоленская,NaN,200.0,200.0/116.0/4.0,5,16,NaN,"4, Оба варианта",Евроремонт,Квартира,5/16,500000,500000,Можно с детьми
3,272197456,"Москва, переулок Плотников, 21С1",Смоленская,NaN,170.0,170.0/95.0/17.0,5,6,14-42-20-19,"4, Оба варианта",Евроремонт,Квартира,5/6,400000,400000,Можно с животными
4,273614615,"Москва, улица Новый Арбат, 15",Арбатская,NaN,58.0,58.0/38.0/5.0,12,26,20 18,2,Евроремонт,Квартира,"12/26, Панельный",225000,225000,NaN


In [11]:
metro = pd.read_csv('metro - metro.csv', sep=',').rename(columns={'Station':'Станция'})
def func(s):
    s = s.replace('ё','е')
    s = s.replace('  ',' ')
    if s == 'Марк':
        s = 'Алтуфьево'
    elif s == 'Перерва':
        s = 'Братиславская'
    elif s.lower() == 'библиотека им. ленина':
        s = 'библиотека имени ленина'
    elif s.lower() == 'бескудниково':
        s = 'селигерская'
    elif s.lower() == 'гражданская':
        s = 'дмитровская'
    return s.lower().strip()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19391 entries, 0 to 19390
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        19391 non-null  int64  
 1   Адрес                     19391 non-null  object 
 2   Станция                   19391 non-null  object 
 3   Серия дома                2088 non-null   object 
 4   Общая площадь             19391 non-null  float64
 5   Площадь                   19391 non-null  object 
 6   Этаж                      19391 non-null  int32  
 7   Этажность дома            19391 non-null  int32  
 8   Площадь комнат, м2        12306 non-null  object 
 9   Количество комнат         18868 non-null  object 
 10  Ремонт                    16944 non-null  object 
 11  Тип                       19391 non-null  object 
 12  Дом                       19391 non-null  object 
 13  Цена за месяц             19391 non-null  int64  
 14  Залог 

In [12]:
#координаты метро

metro = metro.drop_duplicates('Станция').loc[:,['Станция','coor']]
metro.loc[:,'Станция'] = metro.loc[:,'Станция'].map(func)
df.loc[:,'Станция'] = df.loc[:,'Станция'].map(func)

m = df.merge(metro, how = 'left', on='Станция').dropna(how='all')
df.insert(3, "metro_coord",m['coor'])

In [13]:
#Координаты адреса дома
house = pd.read_csv('coordinates.csv', sep=',')
house = house.rename(columns={'ID  объявления':'ID'}).reset_index(drop=True).drop('Unnamed: 0', axis=1)
me = df.merge(house, how = 'left', on='ID').dropna(how='all')

In [14]:
df.insert(2, "house_coord",me['coor'])

In [15]:
df.insert(5,'house_metro',0)
df.insert(6,'house_kremlin',0)


# Релиз 2.0 Расстояния
1. Расстояние до Кремля от дома: house_kremlin
2. Расстояние от дома до метро: house_metro
3. Если нет координаты дома, то среднее расстоние до метро в районе // ср расстония от района до кремля
4. Если нет координат метро, то макс расстояние до метро + макс расстояние до креля

In [16]:
import geopy
import geopy.distance
import numpy as np
kremlin = (55.7516212, 37.618122044334896)
def leng(coord_1, coord_2=kremlin):
    try:
        temp = coord_1.split(',')
        coord_1 = tuple((float(temp[0][1:]),float(temp[1][:-1])))
        if coord_2 == kremlin:
            return round(geopy.distance.geodesic(coord_1, coord_2).km,3)
        else:
            temp = coord_2.split(',')
            coord_2 = tuple((float(temp[0][1:]),float(temp[1][:-1])))
            return round(geopy.distance.geodesic(coord_1, coord_2).km,3)
    except Exception as e:
        return np.NaN

In [17]:
#расстояние от дома до кремля
df['house_kremlin'] = df['house_coord'].map(leng)

In [18]:
#расстояние от дома до метро
df['house_metro'] = [leng(a,b) for a, b in df.loc[:,['house_coord','metro_coord']].itertuples(index=False)]

In [19]:
#расстояние от метро (района) до кремля
df.insert(6,'metro_kremlin',0)
df['metro_kremlin'] = df['metro_coord'].map(leng)

In [20]:
df.drop(['house_coord', 'metro_coord'], axis= 1, inplace=True)

# Релиз 2.0 Заполнение пропусков

In [22]:
# Обработка колонки Площадь комнат

def process_area(x):
    if pd.notna(x):
        numbers = [float(num) for num in re.findall(r'\d+\.\d+|\d+', x)]
        return numbers if numbers else [float(x)]
    else:
        return 0

df['Площадь комнат, м2'] = df['Площадь комнат, м2'].apply(process_area)


In [24]:
# Обработка колонки Количество комнат: приведение к int и заполнение NaN значениями из колонки Площадь комнат, м2, а затем модой
df['Количество комнат'] = df['Количество комнат'].str.extract('(\d+)')

# df['Количество комнат'] = df.apply(lambda row: len(row['Площадь комнат, м2']) if pd.isna(row['Количество комнат']) and row['Площадь комнат, м2'] != 0 else row['Количество комнат'], axis=1)
# mode = df['Количество комнат'].mode()
# df['Количество комнат'] = df['Количество комнат'].fillna(mode)


In [51]:
def func(x):
    try:
        return int(x)
    except:
        return np.NaN
df['rooms'] = df['Количество комнат'].map(func)
df['rooms'].fillna(2, inplace=True)

In [ ]:
df.drop(['Серия дома', 'Площадь', 'Площадь комнат, м2', 'Тип'], axis=1, inplace=True)


In [54]:
df['Можно с детьми/животными'].value_counts()

Можно с детьми/животными
Можно с детьми                       8373
Можно с детьми, Можно с животными    5981
Можно с животными                     206
Name: count, dtype: int64

In [55]:
def process_children(x):
    try:
        if 'детьми' in x:
            return 1
        else:
            return 0
    except:
        return 0
df['Children'] = df['Можно с детьми/животными'].map(process_children)

def process_animal(x):
    try:
        if 'животными' in x:
            return 1
        else:
            return 0
    except:
        return 0

df['Animal'] = df['Можно с детьми/животными'].map(process_animal)

In [56]:
df.drop('Можно с детьми/животными', axis=1, inplace=True)

In [57]:
df['Ремонт'].value_counts()

Ремонт
Косметический    7157
Евроремонт       6908
Дизайнерский     2746
Без ремонта       133
Name: count, dtype: int64

In [58]:
def f1(x):
    try:
        if 'Евро' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f2(x):
    try:
        if 'Дизайн' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f3(x):
    try:
        if 'Косм' in x:
            return 1
        else:
            return 0
    except:
        return 0
def f4(x):
    try:
        if 'Без' in x:
            return 1
        else:
            return 0
    except:
        return 0

In [59]:
df['euro'] = df['Ремонт'].map(f1)
df['design'] = df['Ремонт'].map(f2)
df['cosmetic'] = df['Ремонт'].map(f3)
df['no_rem'] = df['Ремонт'].map(f4)

In [60]:
df.drop('Ремонт', axis=1, inplace=True)

In [63]:
df.drop('Количество комнат', axis=1, inplace=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19391 entries, 0 to 19390
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              19391 non-null  int64  
 1   Адрес           19391 non-null  object 
 2   Станция         19391 non-null  object 
 3   house_metro     14669 non-null  float64
 4   metro_kremlin   18884 non-null  float64
 5   house_kremlin   15026 non-null  float64
 6   Общая площадь   19391 non-null  float64
 7   Этаж            19391 non-null  int32  
 8   Этажность дома  19391 non-null  int32  
 9   Дом             19391 non-null  object 
 10  Цена за месяц   19391 non-null  int64  
 11  Залог           19391 non-null  int64  
 12  rooms           19391 non-null  float64
 13  Children        19391 non-null  int64  
 14  Animal          19391 non-null  int64  
 15  euro            19391 non-null  int64  
 16  design          19391 non-null  int64  
 17  cosmetic        19391 non-null 

In [80]:
df['type_house'] = [row.split(', ')[-1] if len(row.split(', ')[-1]) > 5 else 'Неизвестно' for row in df['Дом']] 
filter = (df['type_house'].isin(('старый фонд', 'Щитовой', 'Деревянный')))
df = df[~filter]


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19318 entries, 0 to 19390
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              19318 non-null  int64  
 1   Адрес           19318 non-null  object 
 2   Станция         19318 non-null  object 
 3   house_metro     14608 non-null  float64
 4   metro_kremlin   18812 non-null  float64
 5   house_kremlin   14964 non-null  float64
 6   Общая площадь   19318 non-null  float64
 7   Этаж            19318 non-null  int32  
 8   Этажность дома  19318 non-null  int32  
 9   Дом             19318 non-null  object 
 10  Цена за месяц   19318 non-null  int64  
 11  Залог           19318 non-null  int64  
 12  rooms           19318 non-null  float64
 13  Children        19318 non-null  int64  
 14  Animal          19318 non-null  int64  
 15  euro            19318 non-null  int64  
 16  design          19318 non-null  int64  
 17  cosmetic        19318 non-null  int6

In [82]:
df.to_csv('data.csv')

# Релиз 2.0 Районы

In [16]:
df = pd.read_csv('data.csv').drop('Unnamed: 0',axis=1).rename(columns={'Адрес':'adress','Станция':'Station','Этаж':'floor','Этажность дома':'height','Цена за месяц':'price','Залог':'deposit' })
df

,ID,adress,Station,house_metro,metro_kremlin,house_kremlin,Общая площадь,floor,height,Дом,price,deposit,rooms,Children,Animal,euro,design,cosmetic,no_rem,type_house
0,271271157,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,200.0,5,16,"5/16, Монолитный",500000,500000,4.0,1,1,0,1,0,0,Монолитный
1,271634126,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,198.0,5,16,"5/16, Монолитно-кирпичный",500000,500000,4.0,1,0,0,1,0,0,Монолитно-кирпичный
2,271173086,"Москва, улица Новый Арбат, 27",смоленская,0.390,2.224,2.410,200.0,5,16,5/16,500000,500000,4.0,1,0,1,0,0,0,Неизвестно
3,272197456,"Москва, переулок Плотников, 21С1",смоленская,NaN,2.224,NaN,170.0,5,6,5/6,400000,400000,4.0,0,1,1,0,0,0,Неизвестно
4,273614615,"Москва, улица Новый Арбат, 15",арбатская,1.469,0.593,2.059,58.0,12,26,"12/26, Панельный",225000,225000,2.0,0,0,1,0,0,0,Панельный
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19313,215565511,"Москва, Боровское шоссе, 2к7, ш. Боровское (3 ...",говорово,NaN,18.647,NaN,35.0,10,14,10/14,42000,42000,2.0,0,0,1,0,0,0,Неизвестно
19314,274654844,"Москва, Производственная улица, 8к1, ш. Боровс...",солнцево,NaN,16.330,NaN,38.7,5,18,"5/18, Монолитный",45000,45000,1.0,0,0,1,0,0,0,Монолитный
19315,268679909,"Москва, Боровский проезд, 11",солнцево,NaN,16.330,NaN,43.1,5,5,"5/5, Кирпичный",50000,50000,2.0,1,0,0,1,0,0,Кирпичный
19316,274807525,"Москва, улица Богданова, 6к1, ш. Боровское (3 ...",солнцево,NaN,16.330,NaN,52.5,8,23,"8/23, Монолитный",55000,50000,2.0,0,0,1,0,0,0,Монолитный


In [10]:
dist = pd.read_csv('districts.csv', sep = ';').loc[1:,:]

In [50]:
def f(s):
    s = s.strip()
    s = s.lower()
    return s
dist['Station']=dist['Station'].map(f)
dist = dist.loc[:,['Station','Line','AdmArea','District']].drop_duplicates('Station')
df = df.merge(dist, on='Station',how='left').dropna(how='all')

In [55]:
#чистка
df = df.drop(['adress','Дом'],axis=1).rename(columns={'Station':'station','Общая площадь':'sq','Children':'child','Animal':'animal','Line':'line','AdmArea':'area','District':'dist'})

In [58]:
df.to_csv('data.csv')

In [64]:
df = df.dropna(subset=['area','metro_kremlin'])

In [83]:
data = df.house_metro
data.sort_values(ascending=False)

filter = (df['house_metro'] > 20)
# df = df[~filter]

In [86]:
filter.value_counts()

house_metro
False    16584
True       619
Name: count, dtype: int64

In [85]:
df.house_metro.median()

1.5875

In [97]:
def func(a,b):
    if np.isnan(b):
        return a
    return b

x = df.copy()

x['house_kremlin'] =  [func(a,b) for a, b in df.loc[:,['metro_kremlin','house_kremlin']].itertuples(index=False)]
df = x

In [104]:
df.drop(['house_metro'],axis=1).to_csv('data_check.csv')

# Релиз 3.0

In [108]:
# df = df.drop(['station','house_metro'],axis=1)
df.to_csv('data.csv')

In [112]:
df = df.drop(['line','dist'],axis=1)
df= df.drop(['dist'],axis=1)

In [113]:
df.groupby('type_house')

,ID,metro_kremlin,house_kremlin,sq,floor,height,price,deposit,rooms,child,animal,euro,design,cosmetic,no_rem,type_house,area
0,271271157,2.224,2.410,200.0,5,16,500000,500000,4.0,1,1,0,1,0,0,Монолитный,Центральный административный округ
1,271634126,2.224,2.410,198.0,5,16,500000,500000,4.0,1,0,0,1,0,0,Монолитно-кирпичный,Центральный административный округ
2,271173086,2.224,2.410,200.0,5,16,500000,500000,4.0,1,0,1,0,0,0,Неизвестно,Центральный административный округ
3,272197456,2.224,2.224,170.0,5,6,400000,400000,4.0,0,1,1,0,0,0,Неизвестно,Центральный административный округ
4,273614615,0.593,2.059,58.0,12,26,225000,225000,2.0,0,0,1,0,0,0,Панельный,Центральный административный округ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19313,215565511,18.647,18.647,35.0,10,14,42000,42000,2.0,0,0,1,0,0,0,Неизвестно,Западный административный округ
19314,274654844,16.330,16.330,38.7,5,18,45000,45000,1.0,0,0,1,0,0,0,Монолитный,Западный административный округ
19315,268679909,16.330,16.330,43.1,5,5,50000,50000,2.0,1,0,0,1,0,0,Кирпичный,Западный административный округ
19316,274807525,16.330,16.330,52.5,8,23,55000,50000,2.0,0,0,1,0,0,0,Монолитный,Западный административный округ


In [135]:
# выбросы по расстоянию
df = df.sort_values(by='house_kremlin', ascending=False).tail(17150)

In [144]:
# выбросы по площади
df = df.sort_values(by='sq', ascending=False).tail(17125)

In [152]:
# выбросы по цене
df = df.sort_values(by='price', ascending=False).head(17100)

In [154]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()

In [165]:
df["make_code"] = ord_enc.fit_transform(df[["area"]])
df["type_h_skl"] = ord_enc.fit_transform(df[["type_house"]])
df[["area", "make_code",'type_house','type_h_skl']].tail(11)

In [171]:
#чистка
# df = df.drop(['area','type_house'],axis=1)
df = df.rename(columns={'make_code':'area_code_skl'})

In [172]:
df.head()

,ID,metro_kremlin,house_kremlin,sq,floor,height,price,deposit,rooms,child,animal,euro,design,cosmetic,no_rem,area_code_skl,type_h_skl
3987,273806652,1.756,1.768,358.0,1,3,3000000,3000000,5.0,1,1,0,1,0,0,6.0,1.0
4032,274270529,0.853,1.768,358.0,1,6,3000000,3000000,6.0,1,1,0,1,0,0,6.0,1.0
12428,268954105,1.315,1.768,358.0,1,6,3000000,3000000,6.0,1,1,0,0,0,0,6.0,2.0
12429,268953362,2.334,1.768,358.0,1,6,3000000,3000000,5.0,1,1,0,0,0,0,6.0,2.0
12427,251335838,2.334,2.031,320.0,2,8,2500000,2500000,4.0,1,0,0,0,0,0,6.0,4.0


In [174]:
df.to_csv('data_skl.csv')